In [1]:
import os
import json
import pandas as pd
import numpy as np
from pathlib import Path

experiment_dir = Path("/gpfs/Labs/Uzun/DATA/PROJECTS/2024.SINGLE_CELL_GRN_INFERENCE.MOELLER/experiments")
training_data_cache_dir = Path("/gpfs/Labs/Uzun/SCRIPTS/PROJECTS/2024.SINGLE_CELL_GRN_INFERENCE.MOELLER/data/training_data_cache")

experiment_dict = {
    "Initial Settings": experiment_dir / "mESC_no_scale_linear" / "model_training_192_10k_metacells",
    "large_neighborhood_count_filter": experiment_dir / "mESC_large_neighborhood_count_filter" / "chr19" / "model_training_001",
    "large_neighborhood": experiment_dir / "mESC_large_neighborhood" / "chr19" / "model_training_001",
    "small_neighborhood": experiment_dir / "mESC_small_neighborhood" / "chr19" / "model_training_001",
    "small_neighborhood_high_self_weight": experiment_dir / "mESC_small_neighborhood_high_self_weight" / "chr19" / "model_training_001",
    "slower_dist_decay": experiment_dir / "mESC_slower_dist_decay" / "chr19" / "model_training_001",
    "max_dist_bias": experiment_dir / "mESC_max_dist_bias" / "chr19" / "model_training_002",
    "slow_decay_max_dist": experiment_dir / "mESC_slow_decay_max_dist" / "chr19" / "model_training_001",
    "filter_lowest_ten_pct": experiment_dir / "mESC_filter_lowest_ten_pct" / "chr19" / "model_training_003",
    "lower_peak_threshold": experiment_dir / "mESC_lower_peak_threshold" / "chr19" / "model_training_001",
    "no_filter_to_nearest_gene": experiment_dir / "mESC_no_filter_to_nearest_gene" / "chr19" / "model_training_001",
    "smaller_window_size": experiment_dir / "mESC_smaller_window_size" / "chr19" / "model_training_001",
    "larger_window_size": experiment_dir / "mESC_larger_window_size" / "chr19" / "model_training_001",
    "lower_max_peak_dist": experiment_dir / "mESC_lower_max_peak_dist" / "chr19" / "model_training_001",
    # "higher_max_peak_dist": experiment_dir / "mESC_higher_max_peak_dist" / "chr19" / "model_training_001",
    "test_new_pipeline": experiment_dir / "mESC_test_new_pipeline" / "chr19" / "model_training_001",
    # "slow_decay_filter_ten_pct": experiment_dir / "mESC_slow_decay_filter_ten_pct" / "chr19" / "model_training_001"
}

experiment_training_data_dict = {
    "Initial Settings": training_data_cache_dir / "mESC_no_scale_linear",
    "large_neighborhood_count_filter": training_data_cache_dir / "mESC_large_neighborhood_count_filter",
    "large_neighborhood": training_data_cache_dir / "mESC_large_neighborhood",
    "small_neighborhood": training_data_cache_dir / "mESC_small_neighborhood",
    "small_neighborhood_high_self_weight": training_data_cache_dir / "mESC_small_neighborhood_high_self_weight",
    "slower_dist_decay": training_data_cache_dir / "mESC_slower_dist_decay",
    "max_dist_bias": training_data_cache_dir / "mESC_max_dist_bias",
    "slow_decay_max_dist": training_data_cache_dir / "mESC_slow_decay_max_dist",
    "filter_lowest_ten_pct": training_data_cache_dir / "mESC_filter_lowest_ten_pct",
    "lower_peak_threshold": training_data_cache_dir / "mESC_lower_peak_threshold",
    "no_filter_to_nearest_gene": training_data_cache_dir / "mESC_no_filter_to_nearest_gene",
    "smaller_window_size": training_data_cache_dir / "mESC_smaller_window_size",
    "larger_window_size": training_data_cache_dir / "mESC_larger_window_size",
    "lower_max_peak_dist": training_data_cache_dir / "mESC_lower_max_peak_dist",
    # "higher_max_peak_dist": training_data_cache_dir / "mESC_higher_max_peak_dist",
    "test_new_pipeline": training_data_cache_dir / "mESC_test_new_pipeline",
    # "slow_decay_filter_ten_pct": training_data_cache_dir / "mESC_slow_decay_filter_ten_pct"
}

# Excel/Sheet ordering
GT_ORDER = ["ChIP-Atlas", "RN111", "RN112", "RN114", "RN115", "RN116"]
METHOD_ORDER = ["Gradient Attribution", "TF Knockout"]

def compute_total_windows(cache_dir: Path) -> int:
    total = 0
    for d in cache_dir.iterdir():
        if d.is_dir() and d.name.startswith("chr"):
            with open(cache_dir / d / f"window_map_{d.name}.json", "r") as f:
                window_map = json.load(f)
            total += len(window_map)
    return total

def format_block_columns(prefix: str, gt_order=GT_ORDER):
    """
    Returns the column order for one method block exactly like your sheet:
    Experiment | Mean AUROC/AUPRC | then each GT has AUROC, Per-TF AUROC, AUPRC, Per-TF AUPRC
    """
    cols = [
        (prefix, "Mean", "AUROC"),
        (prefix, "Mean", "AUPRC"),
        (prefix, "Mean", "Per-TF AUROC"),
        (prefix, "Mean", "Per-TF AUPRC"),
    ]
    for gt in gt_order:
        cols += [
            (prefix, gt, "AUROC"),
            (prefix, gt, "Per-TF AUROC"),
            (prefix, gt, "AUPRC"),
            (prefix, gt, "Per-TF AUPRC"),
        ]
    return cols

all_experiments = []

for experiment_name, EXPERIMENT_DIR in experiment_dict.items():
    print(f"\nProcessing experiment: {experiment_name}")

    # --- dataset sizes ---
    with open(EXPERIMENT_DIR / "tf_vocab.json", "r") as f:
        num_tfs = len(json.load(f))
    with open(EXPERIMENT_DIR / "tg_vocab.json", "r") as f:
        num_tgs = len(json.load(f))

    
    total_windows = compute_total_windows(experiment_training_data_dict[experiment_name])

    # --- final R2 ---
    training_results = pd.read_csv(EXPERIMENT_DIR / "training_log.csv")
    final_epoch_results = training_results.iloc[-1]
    final_train_unscaled_r2 = float(final_epoch_results["R2_u"])
    final_train_scaled_r2   = float(final_epoch_results["R2_s"])

    # ============================================================
    # UPDATED PART: pooled vs per-TF loaded separately
    # ============================================================

    # ---------- 1) POOLED overall mean AUROC/AUPRC ----------
    pooled_overall = pd.read_csv(EXPERIMENT_DIR / "method_ranking_by_auroc_pooled.csv", header=0)
    pooled_overall = pooled_overall.rename(columns={"name": "method_name"})
    pooled_overall = pooled_overall[pooled_overall["method_name"].isin(METHOD_ORDER)]
    pooled_overall = pooled_overall.set_index("method_name")[["mean_auroc", "mean_auprc"]]

    # ---------- 2) POOLED per-GT AUROC/AUPRC ----------
    pooled_gt = pd.read_csv(EXPERIMENT_DIR / "per_gt_method_aucs_pooled.csv", header=0)
    pooled_gt = pooled_gt.rename(columns={"name": "method_name"})
    pooled_gt = pooled_gt[pooled_gt["method_name"].isin(METHOD_ORDER)]
    pooled_gt = pooled_gt[pooled_gt["gt_name"].isin(GT_ORDER)]

    pooled_gt_auroc = pooled_gt.pivot(index="method_name", columns="gt_name", values="auroc")
    pooled_gt_auprc = pooled_gt.pivot(index="method_name", columns="gt_name", values="auprc")

    # ---------- 3) PER-TF detailed -> mean across TFs ----------
    per_tf = pd.read_csv(EXPERIMENT_DIR / "per_tf_auroc_auprc_detailed.csv", header=0)

    # Standardize method column name
    if "method" in per_tf.columns and "method_name" not in per_tf.columns:
        per_tf = per_tf.rename(columns={"method": "method_name"})

    per_tf = per_tf[per_tf["method_name"].isin(METHOD_ORDER)].copy()
    per_tf = per_tf[per_tf["gt_name"].isin(GT_ORDER)].copy()

    # Per-GT per-method: mean across TFs
    per_tf_gt_means = (
        per_tf.groupby(["method_name", "gt_name"], as_index=False)
            .agg(
                per_tf_auroc=("auroc", "mean"),
                per_tf_auprc=("auprc", "mean"),
                n_tfs=("tf", "nunique"),
            )
    )

    per_tf_auroc_lookup = per_tf_gt_means.set_index(["method_name", "gt_name"])["per_tf_auroc"].to_dict()
    per_tf_auprc_lookup = per_tf_gt_means.set_index(["method_name", "gt_name"])["per_tf_auprc"].to_dict()

    # Overall per-TF mean across GTs (left mean block "Per-TF AUROC/AUPRC")
    per_tf_overall = (
        per_tf_gt_means.groupby("method_name", as_index=False)
                    .agg(
                        per_tf_mean_auroc=("per_tf_auroc", "mean"),
                        per_tf_mean_auprc=("per_tf_auprc", "mean"),
                    )
                    .set_index("method_name")
    )

    # ---------- 4) Build one row for this experiment ----------
    row = {
        ("Meta", "", "Experiment"): experiment_name,
        ("Meta", "", "num_tfs"): num_tfs,
        ("Meta", "", "num_windows"): total_windows,
        ("Meta", "", "num_tgs"): num_tgs,
        ("Meta", "", "unscaled_r2"): final_train_unscaled_r2,
        ("Meta", "", "scaled_r2"): final_train_scaled_r2,
    }

    for method in METHOD_ORDER:
        # ---- Mean block ----
        # pooled mean AUROC/AUPRC
        row[(method, "Mean", "AUROC")] = (
            float(pooled_overall.loc[method, "mean_auroc"])
            if method in pooled_overall.index else np.nan
        )
        row[(method, "Mean", "AUPRC")] = (
            float(pooled_overall.loc[method, "mean_auprc"])
            if method in pooled_overall.index else np.nan
        )

        # per-TF mean AUROC/AUPRC (mean across TFs, then mean across GTs)
        row[(method, "Mean", "Per-TF AUROC")] = (
            float(per_tf_overall.loc[method, "per_tf_mean_auroc"])
            if method in per_tf_overall.index else np.nan
        )
        row[(method, "Mean", "Per-TF AUPRC")] = (
            float(per_tf_overall.loc[method, "per_tf_mean_auprc"])
            if method in per_tf_overall.index else np.nan
        )

        # ---- Per-GT blocks ----
        for gt in GT_ORDER:
            # pooled per-GT AUROC/AUPRC
            row[(method, gt, "AUROC")] = (
                float(pooled_gt_auroc.loc[method, gt])
                if (method in pooled_gt_auroc.index and gt in pooled_gt_auroc.columns) else np.nan
            )
            row[(method, gt, "AUPRC")] = (
                float(pooled_gt_auprc.loc[method, gt])
                if (method in pooled_gt_auprc.index and gt in pooled_gt_auprc.columns) else np.nan
            )

            # per-TF per-GT means (mean across TFs)
            row[(method, gt, "Per-TF AUROC")] = float(per_tf_auroc_lookup.get((method, gt), np.nan))
            row[(method, gt, "Per-TF AUPRC")] = float(per_tf_auprc_lookup.get((method, gt), np.nan))

    all_experiments.append(row)


# Build final dataframe
final_df = pd.DataFrame(all_experiments)

# Order columns to match your sheet grouping
meta_cols = [
    ("Meta", "", "Experiment"),
    ("Meta", "", "num_tfs"),
    ("Meta", "", "num_windows"),
    ("Meta", "", "num_tgs"),
    ("Meta", "", "unscaled_r2"),
    ("Meta", "", "scaled_r2"),
]

ordered_cols = meta_cols + format_block_columns("Gradient Attribution") + format_block_columns("TF Knockout")
ordered_cols = [c for c in ordered_cols if c in final_df.columns]  # safety

final_df = final_df[ordered_cols]

# Flatten columns for easy Excel export
def flat(c):
    a, b, d = c
    if a == "Meta":
        return d
    # e.g. "Gradient Attribution | RN111 | Per-TF AUROC"
    return f"{a} | {b} | {d}"

final_df.columns = [flat(c) for c in final_df.columns]

print(final_df.head())

# Export
final_df.to_excel("dev/notebooks/excel_chart_ready.xlsx", index=False)
print("Wrote: excel_chart_ready.xlsx")



Processing experiment: Initial Settings

Processing experiment: large_neighborhood_count_filter

Processing experiment: large_neighborhood

Processing experiment: small_neighborhood

Processing experiment: small_neighborhood_high_self_weight

Processing experiment: slower_dist_decay

Processing experiment: max_dist_bias

Processing experiment: slow_decay_max_dist

Processing experiment: filter_lowest_ten_pct

Processing experiment: lower_peak_threshold

Processing experiment: no_filter_to_nearest_gene

Processing experiment: smaller_window_size

Processing experiment: larger_window_size

Processing experiment: lower_max_peak_dist

Processing experiment: test_new_pipeline
                            Experiment  num_tfs  num_windows  num_tgs  \
0                     Initial Settings      639        19727     6852   
1      large_neighborhood_count_filter      396        33940     7297   
2                   large_neighborhood      183         3174     2008   
3                   small_n